# HCIPy SM on non-normalized apertures

In this notebook I am making my segmented mirror class for HCIPy work also for wavelengths other than 1 (m).  
Note: This turned out to be super easy since i just kept forgetting to set a wavelength parameter in `make_focal_grid()`/`make_focal_grid_from_pupil_grid()`... woops :P

I am also extending the aperture generation and `SegmentedMirror()` to work on non-normalized apertures, which takes a bit more effort.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import astropy.units as u
from hcipy import *

os.chdir('../../pastis/')
from config import CONFIG_PASTIS
import util as util
import atlast_imaging as atim

## Instantiate an HCIPy SM

In [ ]:
# Parameters
npix = 1024
wvln = 638e-9      # wavelength in meters
lamD = 15
samp = 4
diam = 15.    # telescope diameter in meters: 1 if norm=True, 15 if norm=False
norm = False  # whether or not to normalize aperture: True if diam=1, False if not

In [ ]:
# HCIPy grids and propagator
pupil_grid = make_pupil_grid(dims=npix, diameter=diam)
focal_grid = make_focal_grid_from_pupil_grid(pupil_grid, samp, lamD, wavelength=wvln)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

In [ ]:
# Generate an ATLAST pupil
aper, seg_pos = atim.get_atlast_aperture(normalized=norm)
aper = evaluate_supersampled(aper, pupil_grid, 2)

imshow_field(aper)
plt.title('Aperture')

In [ ]:
# Instantiate the segmented mirror
sm = atim.SegmentedMirrorAtlast(aper, seg_pos)

In [ ]:
# Make a pupil plane wavefront from aperture
wf = Wavefront(aper, wavelength=wvln)
imshow_field(wf.intensity)
plt.title('Wavefront intensity')

## Propagate image

Here is where the trouble starts.

In [ ]:
im_ref = prop(sm(wf))

plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_ref.intensity))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im_ref.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

## Take a step back and do this without an SM

In [ ]:
# Parameters
npix = 1024
wvln = 638e-9     # m
lamD = 15
samp = 4
diam = 15.    # m

In [ ]:
# Grids and propagator
pgrid = make_pupil_grid(dims=npix, diameter=diam,)
fgrid = make_focal_grid_from_pupil_grid(pgrid, samp, lamD, wavelength=wvln)
prop_easy = FraunhoferPropagator(pgrid, fgrid)

In [ ]:
# Generate circular pupil
circ = circular_aperture(diameter=diam)
circ = evaluate_supersampled(circ, pgrid, 2)

imshow_field(circ)

In [ ]:
# Propagate to image plane
im = prop_easy(Wavefront(circ, wavelength=wvln))

imshow_field(np.log10(im.intensity))

## Aperture generation for normalize=False

Need to first make sure that the aperture generation yields a correct NRP vector when I don't normalize the aperture.

In [ ]:
# normalized=True
aper1, seg_pos1 = atim.get_atlast_aperture(normalized=True, write_to_disk=True, outDir='/Users/ilaginja')
print(seg_pos1.points)

In [ ]:
# normalized=False
aper2, seg_pos2 = atim.get_atlast_aperture(normalized=False, write_to_disk=True, outDir='/Users/ilaginja')
print(seg_pos2.points)

The aperture generation works with non-normalized apertures now.